In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import os
from pathlib import Path
import shutil
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import nltk
from gensim.models.keyedvectors import KeyedVectors
nltk.download('stopwords')
from nltk.corpus import stopwords
import gc     

[nltk_data] Downloading package stopwords to /Users/misha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
dataset_dir = '/Users/misha/Desktop/My_Work/Datasets/DAIC_WOZ'
label_dir = '/Users/misha/Desktop/My_Work/Datasets/DAIC_WOZ/labels/'
data_dir = '/Users/misha/Desktop/My_Work/Datasets/DAIC_WOZ/data/'
splitted_dataset_dir = '/Users/misha/Desktop/My_Work/Datasets/DAIC_WOZ/splitted_dataset/'

In [4]:
if not os.path.exists(splitted_dataset_dir+'train_data'):
    os.makedirs(splitted_dataset_dir+'train_data')

if not os.path.exists(splitted_dataset_dir+'dev_data'):
    os.makedirs(splitted_dataset_dir+'dev_data')
    
if not os.path.exists(splitted_dataset_dir+'test_data'):
    os.makedirs(splitted_dataset_dir+'test_data')

In [6]:
# dev = np.array(pd.read_csv(label_dir+'train_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
# train = np.array(pd.read_csv(label_dir+'test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
# test = np.array(pd.read_csv(label_dir+'dev_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]

train = np.array(pd.read_csv(label_dir+'train_split.csv',delimiter=',',encoding='utf-8'))
dev = np.array(pd.read_csv(label_dir+'dev_split.csv',delimiter=',',encoding='utf-8'))
test = np.array(pd.read_csv(label_dir+'test_split.csv',delimiter=',',encoding='utf-8'))

In [7]:
print('train_size: ', len(train))
print('dev_size: ', len(dev))
print('test_size: ', len(test))

train_size:  163
dev_size:  56
test_size:  56


In [97]:
# for i in range(len(train)):
#     source_dir = data_dir + str(int(train[i][0])) + "_P"
#     target_dir = splitted_dataset_dir + 'train_data/' + str(int(train[i][0])) + "_P"
#     shutil.move(source_dir, target_dir)

In [98]:
# for i in range(len(dev)):
#     source_dir = data_dir + str(int(dev[i][0])) + "_P"
#     target_dir = splitted_dataset_dir + 'dev_data/' + str(int(dev[i][0])) + "_P"
#     shutil.move(source_dir, target_dir)

In [99]:
# for i in range(len(test)):
#     source_dir = data_dir + str(int(test[i][0])) + "_P"
#     target_dir = splitted_dataset_dir + 'test_data/' + str(int(test[i][0])) + "_P"
#     shutil.move(source_dir, target_dir)

In [8]:
dataset1 = dev
dataset2 = test
dataset3 = train
dataset = np.concatenate((dataset1, np.concatenate((dataset2, dataset3))))

In [9]:
dataset

array([[300, 'male', 0, 2, 0, 25],
       [301, 'male', 0, 3, 0, 17],
       [306, 'female', 0, 0, 0, 21],
       ...,
       [702, 'male', 0, 0, 0, 19],
       [703, 'male', 0, 8, 0, 28],
       [707, 'male', 0, 1, 0, 23]], dtype=object)

In [10]:
positive = []
negative = []

In [11]:
for i in range(len(dataset)):
    if dataset[i][0]<=310 and dataset[i][0]>=300:
        if(dataset[i][1] == 0):
            positive.append(int(dataset[i][0]))
        else:
            negative.append(int(dataset[i][0]))

    elif dataset[i][0]<=355 and dataset[i][0]>=345:
        if(dataset[i][1] == 0):
            positive.append(int(dataset[i][0]))
        else:
            negative.append(int(dataset[i][0]))

In [12]:
for i in range(len(dataset)):
        if(dataset[i][2] == 0):
            positive.append(int(dataset[i][0]))
        else:
            negative.append(int(dataset[i][0]))


In [13]:
len(negative)

88

In [14]:
len(positive)

209

In [15]:
print("Text Results: ")

countPos = 0
dataset = np.concatenate((dataset1, np.concatenate((dataset2, dataset3))))
def checkPosNeg(dataset, index):
    for i in range(0, len(dataset)):
        if(dataset[i][0] == index):
            return dataset[i][2]
    return 0

Data = []
Y = []

Data_test = []
Y_test = []
index = -1
for i in range(0, len(dataset3)):
    val = checkPosNeg(dataset, dataset3[i][0])
    if(val == 0 and countPos>38):
        continue
    Y.append(val)
    index+=1
    if(Y[index] == 0):
        countPos+=1
    
    fileName = "/Users/misha/Desktop/My_Work/Datasets/DAIC_WOZ/splitted_dataset/train_data/" + \
                str(int(dataset3[i][0])) + '_P/' + str(int(dataset3[i][0])) + "_Transcript.csv"
    Data.append(np.array(pd.read_csv(fileName,delimiter=',',encoding='utf-8'))[:, 2:3])
    
for i in range(0, len(dataset1)):
    val = checkPosNeg(dataset, dataset1[i][0])
    if(val == 0):
        continue
    Y.append(val)
    fileName = "/Users/misha/Desktop/My_Work/Datasets/DAIC_WOZ/splitted_dataset/dev_data/" + \
                str(int(dataset1[i][0])) + '_P/' + str(int(dataset1[i][0])) + "_Transcript.csv"
    Data.append(np.array(pd.read_csv(fileName,delimiter=',',encoding='utf-8'))[:, 2:3])
    
for i in range(0, len(dataset2)):
    Y_test.append(checkPosNeg(dataset, dataset2[i][0]))
    fileName = "/Users/misha/Desktop/My_Work/Datasets/DAIC_WOZ/splitted_dataset/test_data/" + \
                str(int(dataset2[i][0])) + '_P/' + str(int(dataset2[i][0])) + "_Transcript.csv"
    Data_test.append(np.array(pd.read_csv(fileName,delimiter=',',encoding='utf-8'))[:, 2:3])


Text Results: 


In [16]:
Y = np.array(Y)
Data2 = []

Data2_test = []
Y_test = np.array(Y_test)

for i in range(0, len(Data)):
    script = []
    for k in range(1, len(Data[i])):
        if(Data[i][k][0] == "Participant"):
            script.append(Data[i][k][1])
    Data2.append(script)

for i in range(0, len(Data_test)):
    script = []
    for k in range(1, len(Data_test[i])):
        if(Data_test[i][k][0] == "Participant"):
            script.append(Data_test[i][k][1])
    Data2_test.append(script)
        
Data2 = np.array(Data2)
Data2_test = np.array(Data2_test)


In [17]:
Data2_test.shape

(56, 0)

In [19]:
model = KeyedVectors.load_word2vec_format('/Users/misha/Desktop/My_Work/Datasets/GoogleNews-vectors-negative300.bin', binary=True)

stop_words = set(stopwords.words('english'))


def remove_StopWOrds(sentence):
    filtered_sentence = [] 
    for w in sentence: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    
    return filtered_sentence

def checkAcc(Y_pred, Y_test):
    correct = 0
    for i in range(len(Y_pred)):
        if(Y_pred[i] == Y_test[i]):
            correct+=1
    
    return float(correct)/len(Y_pred)


In [20]:
Data2 = Data
Data2_test = Data_test
Data2 = np.array(Data2)
Data2_test = np.array(Data2_test)

/tmp/ipykernel_66691/1802328726.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Data2 = np.array(Data2)
/tmp/ipykernel_66691/1802328726.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Data2_test = np.array(Data2_test)


In [21]:
max_num_words = 15
max_num_sentence = 200

#train_data
finalMatrix = np.zeros((Data2.shape[0], max_num_sentence, max_num_words))
print(finalMatrix.shape)
for k in range(Data2.shape[0]):
    for i in range(min(max_num_sentence, len(Data2[k]))):
        try:
            lst = Data2[k][i].tolist()
            sentence = ''.join(str(elem) for elem in lst)
            sentence = sentence.split(" ")
#             print('sentence1: ', sentence)
        except:
            continue
        sentence = remove_StopWOrds(sentence)
#         print('sentence2: ', sentence)
        for j in range(min(max_num_words, len(sentence))):
            try:
                finalMatrix[k][i][j] = np.average(np.array(model[sentence[j]]))
#                 print(' finalMatrix ', finalMatrix[k][i][j])
            except:
                continue
                
X = np.zeros((len(Data2), max_num_sentence*max_num_words))

for i in range(len(finalMatrix)):
    X[i] = finalMatrix[i].flatten().reshape(1, -1)


(88, 200, 15)


In [22]:
Data2_test

array([array([['okay'],
              ["there she is coming to go ahead and Shrink her back down while I continue setting something's out okay"],
              [' when she says goodbye'],
              [' okay'],
              [' yes'],
              [" I'm fine"],
              [' Mexico'],
              [' the weather'],
              [' the crime'],
              [" I don't know what that means"],
              [' write poetry'],
              [' nothing'],
              [' mr. music'],
              [' Burger'],
              [' about a week ago my husband over cigarettes'],
              [" she wants to stop and I don't"],
              [' I dropped out when I was in 11th grade'],
              [" I'm on the Floyd"],
              [' I would like to become a famous poetry writer'],
              [' yeah'],
              [' no'],
              [' so expensive'],
              [' when I was little I used to play with my friends and we should have fun'],
              [" I'm very clo

In [23]:
#Test_data
finalMatrix_test = np.zeros((Data2_test.shape[0], max_num_sentence, max_num_words))
print(finalMatrix_test.shape)
for k in range(Data2_test.shape[0]):
    for i in range(min(max_num_sentence, len(Data2_test[k]))):
        try:
            lst = Data2_test[k][i].tolist()
            sentence = ''.join(str(elem) for elem in lst)
            sentence = sentence.split(" ")
        except:
            continue
        sentence = remove_StopWOrds(sentence)
        for j in range(min(max_num_words, len(sentence))):
            try:
                finalMatrix_test[k][i][j] = np.average(np.array(model[sentence[j]]))
            except:
                continue
                
X_test = np.zeros((len(Data2_test), max_num_sentence*max_num_words))

for i in range(len(finalMatrix_test)):
    X_test[i] = finalMatrix_test[i].flatten().reshape(1, -1)

print(finalMatrix_test.shape)

# Data2 = []
# Data2_test = []
gc.collect()

(56, 200, 15)
(56, 200, 15)


489

In [24]:
#Random Forest Classifier
clf1 = RandomForestClassifier()
clf1.fit(X,Y)
Y_pred_rf_1 = clf1.predict(X_test)
print("Random forests: ")
print(classification_report(Y_test,Y_pred_rf_1))
print("Train: "+ str(clf1.score(X,Y)))
print("Test: " + str(clf1.score(X_test,Y_test)))

Random forests: 
              precision    recall  f1-score   support

           0       0.56      0.13      0.21        39
           1       0.28      0.76      0.41        17

    accuracy                           0.32        56
   macro avg       0.42      0.45      0.31        56
weighted avg       0.47      0.32      0.27        56

Train: 1.0
Test: 0.32142857142857145


In [174]:
#SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X, Y)
Y_pred_svm_1 = classifier.predict(X_test)
print("SVM: ")
print(classification_report(Y_test,Y_pred_svm_1))
print("Train: "+ str(classifier.score(X,Y)))
print("Test: " + str(classifier.score(X_test,Y_test)))

SVM: 
              precision    recall  f1-score   support

           0       1.00      0.03      0.05        39
           1       0.31      1.00      0.47        17

    accuracy                           0.32        56
   macro avg       0.65      0.51      0.26        56
weighted avg       0.79      0.32      0.18        56

Train: 1.0
Test: 0.32142857142857145


In [175]:
y_pred_train_svm_1 = classifier.predict(X)
y_pred_train_rf_1 = clf1.predict(X)

In [179]:
#Audio Features 
print("Audio Results: ")

def makedata(X):
    for i in range(X.shape[0]):
        if(X[i,1] == 0):
            X[i,0] = 0
            for j in range(7):
                X[i,j+1] = 0
    X = np.array(X)
    X = np.average(X, axis = 0)
    X = np.array(X.reshape(1, -1))
    return X
 
X_train = np.zeros((1,74))
Y_train = []

X_test = np.zeros((1,74))
#Y_test = []

for i in range(len(dataset2)):
    data = pd.read_csv('/Users/misha/Desktop/My_Work/Datasets/DAIC_WOZ/splitted_dataset/test_data/'+str(int(dataset2[i][0]))+"_COVAREP.csv",header = None)
    X_temp = data.iloc[:,:].values
    X_temp = makedata(X_temp)
    X_test = np.concatenate((X_test,X_temp),0)
    #Y_test.append(dataset2[i][1])
    

X_test = np.delete(X_test,0,0)
#Y_test = np.array(Y_test)
count_0 = 0
count_1 = 0

for i in range(len(dataset3)):
    data = pd.read_csv('/content/drive/My Drive/MCA Dataset/train_data/'+str(int(dataset3[i][0]))+"_COVAREP.csv",header = None)
    if(dataset3[i][1] == 0):
      count_0 +=1
      if(count_0<=39):
        X_temp = data.iloc[:,:].values
        X_temp = makedata(X_temp)
        X_train = np.concatenate((X_train,X_temp),0)
        Y_train.append(dataset3[i][1])
    else:
      count_1 +=1
      if(count_1<=28):
        X_temp = data.iloc[:,:].values
        X_temp = makedata(X_temp)
        X_train = np.concatenate((X_train,X_temp),0)
        Y_train.append(dataset3[i][1])

Audio Results: 
